We hebben alle `nvt` en `?` kolomen verwijderd. Dit hebben we gedaan omdat er tegen ons gezegd is dat deze niet interresand zijn. Daarnaast hebben we ook de regels met prioriteit `9` verwijderd. Dit komt omdat deze regels adimistratief zijn en geen functie herstel bevatten.

Vervolgens hebben we nog een aantal kolomen ook nog verwijderd, die worden hieronder verder toegeligt:

`stm_pplg_van` en `stm_pplg_naar`. Wij hebben hier niks aan. Er worden namelijk zowel getallen als strings door elkaar gebruikt. Dit is niet handig voor zo goed als elke model. Daarom hebben we deze kolomen verwijderd.

Ook hebben wij de kolommen `stm_equipm_omschr_gst`, `stm_equipm_omschr_mld`, `stm_sap_meldtekst`, `stm_sap_meldtekst_lang`, `stm_oorz_tkst`, `stm_oorz_tekst_kort`, `stm_schade_verhaalb_jn` verwijderd. Dit komt omdat deze beschrijvingen zijn en dus niet bruikbaar zijn voor een model.

Daarnaast hebben we ook nog de volgende kolommen verwijderd: `stm_equipm_nr_gst`,  `stm_equipm_soort_gst`, `#stm_sap_meldnr`, `stm_equipm_nr_mld`, `stm_equipm_soort_mld`. Dit komt omdat deze kolommen niet relevant zijn voor ons model omdat ze geen informatie toevoegen.

`stm_vl_post` is verwijderd omdat hier veel values missen en de *informatie al hebben in andere kolommen* (niet excact dezelfde informatie maar `stm_vl_post` komt wel in de buurt).

Wij hebben de kolomen  `stm_fh_tijd`, `stm_fh_dd` ook verwijderd omdat je hiermee anders de target variabele kan voorspellen doormiddel van begin tijd - funtie herstel tijd = duur. Dit is niet de bedoeling. **(Mogelijk niet waar?)**

Wij hebben als feuture variabelen de volgende gekozen: `stm_geo_mld`, `stm_prioriteit`, `stm_oorz_code`, `stm_contractgeb_gst`

Over de volgende feuture variabelen twijfelen we nog of ze nuttig zijn: `stm_mon_nr`, `stm_sap_meld_ddt`, `stm_km_van_mld`, `stm_km_tot_mld`. Hierom gaan we aan deze extra aandacht besteden door ze 1 voor 1 eruit te halen en te kijken of ze het model beinvloeden.

Onze target variabele is `stm_fh_duur` **(mogelijk ook niet waar?)**

In [4]:
import pandas as pd

def clean_data(df):
    # Drop duplicate rows in columns: '#stm_sap_meldnr', 'stm_mon_nr' and 58 other columns
    df = df.drop_duplicates(subset=['#stm_sap_meldnr', 'stm_mon_nr', 'stm_vl_post', 'stm_sap_meld_ddt', 'stm_sap_meldtekst_lang', 'stm_sap_meldtekst', 'stm_geo_mld', 'stm_equipm_nr_mld', 'stm_equipm_soort_mld', 'stm_equipm_omschr_mld', 'stm_km_van_mld', 'stm_km_tot_mld', 'stm_prioriteit', 'stm_aanngeb_ddt', 'stm_oh_pg_gst', 'stm_geo_gst', 'stm_equipm_nr_gst', 'stm_equipm_soort_gst', 'stm_equipm_omschr_gst', 'stm_km_van_gst', 'stm_km_tot_gst', 'stm_oorz_groep', 'stm_oorz_code', 'stm_oorz_tkst', 'stm_fh_ddt', 'stm_fh_status', 'stm_sap_storeind_ddt', 'stm_tao_indicator', 'stm_tao_indicator_vorige', 'stm_tao_soort_mutatie', 'stm_tao_telling_mutatie', 'stm_tao_beinvloedbaar_indicator', 'stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_contractgeb_mld', 'stm_techn_mld', 'stm_contractgeb_gst', 'stm_techn_gst', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd', 'stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur', 'stm_progfh_gw_tijd', 'stm_progfh_gw_teller', 'stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd', 'stm_oorz_tekst_kort', 'stm_schade_verhaalb_jn', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar'])
    # Drop columns: 'stm_pplg_van', 'stm_pplg_naar' and 7 other columns
    df = df.drop(columns=['stm_pplg_van', 'stm_pplg_naar', 'stm_equipm_omschr_mld', 'stm_sap_meldtekst_lang', 'stm_sap_meldtekst', 'stm_equipm_omschr_gst', 'stm_oorz_tkst', 'stm_oorz_tekst_kort', 'stm_schade_verhaalb_jn', 'stm_equipm_nr_gst', 'stm_equipm_soort_gst', '#stm_sap_meldnr', 'stm_vl_post', 'stm_equipm_nr_mld', 'stm_equipm_soort_mld', 'stm_fh_tijd', 'stm_fh_dd'])
    # Drop first row
    df = df.iloc[1:, :]
    # Filter rows based on column: 'stm_prioriteit'
    df = df[df['stm_prioriteit'].isin(["1.0", "2.0", "4.0", "5.0", "8.0"])]
    # Filter rows based on column: 'stm_fh_duur'
    df = df[df['stm_fh_duur'].notna()]
    # Filter not valid integer values in column: 'stm_fh_duur'
    df['stm_fh_duur'] = pd.to_numeric(df['stm_fh_duur'], errors="coerce")
    df.dropna(subset=['stm_fh_duur'], inplace=True)
    # Remove values with a not realistic duration
    df = df[df['stm_fh_duur'] < 43200] # 1 month
    
    # topPerc = df['stm_fh_duur'].quantile(0.99)
    # df = df[df['stm_fh_duur'] < topPerc]
    
    # Split the "contract gebied" code and the "techniekveld" code. Format is always: int:int:letter
    df['stm_oh_pg_gst_tv'] = df['stm_oh_pg_gst'].str[2]
    df['stm_oh_pg_gst_cg'] = df['stm_oh_pg_gst'].str[0:2]
    
    
    df['stm_contractgeb_gst'] = pd.to_numeric(df['stm_contractgeb_gst'], errors="coerce")
    df.dropna(subset=['stm_contractgeb_gst'], inplace=True)
    df = df.astype({'stm_geo_mld':'float', 'stm_prioriteit':'float', 'stm_oorz_code':'float', 'stm_contractgeb_gst':'float', 'stm_mon_nr':'float', 'stm_sap_meld_ddt':'datetime64[ns]', 'stm_km_van_mld':'float', 'stm_km_tot_mld':'float', 'stm_oh_pg_gst_cg':'int'})
    return df

# Loaded variable 'df'
df = pd.read_csv('dataset-prorail-clean-2.csv')

df = clean_data(df.copy())
df.head()

/tmp/ipykernel_701411/1271952366.py:34: DtypeWarning: Columns (0,8,11,12,13,17,20,21,26,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset-prorail-clean-2.csv')


,index2,stm_mon_nr,stm_sap_meld_ddt,stm_geo_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,stm_aanngeb_ddt,stm_oh_pg_gst,stm_geo_gst,...,stm_progfh_in_duur,stm_progfh_gw_tijd,stm_progfh_gw_teller,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_dstrglp_van,stm_dstrglp_naar,stm_oh_pg_gst_tv,stm_oh_pg_gst_cg
13,13,0.0,2006-03-01 19:18:00,112.0,0.0,0.0,5.0,03/01/2006 19:18:00,02B,112.0,...,99999999.0,00:00:00,0.0,24.0,03/01/2006,19:42:00,NaN,NaN,B,2
14,14,0.0,2006-03-01 13:33:00,115.0,0.0,0.0,5.0,03/01/2006 13:33:00,03B,115.0,...,99999999.0,00:00:00,0.0,1.0,03/01/2006,13:34:00,NaN,NaN,B,3
85,87,37.0,2006-10-01 16:11:00,618.0,0.0,0.0,4.0,10/01/2006 16:11:00,24X,NaN,...,99999999.0,00:00:00,0.0,20.0,11/01/2006,10:33:00,EHV,NaN,X,24
237,240,37.0,2006-01-25 08:59:00,564.0,0.0,0.0,2.0,25/01/2006 08:59:00,02S,564.0,...,99999999.0,00:00:00,0.0,4.0,25/01/2006,09:03:00,Gvc,NaN,S,2
304,309,136.0,2006-01-27 18:37:00,561.0,0.0,0.0,2.0,27/01/2006 18:37:00,02S,561.0,...,99999999.0,00:00:00,0.0,117.0,27/01/2006,20:34:00,Laa,NaN,S,2
